# 使ってみたシリーズ

In [34]:
import pandas as pd, numpy as np
#from multiprocessing import Pool
from sklearn.model_selection import train_test_split
# load data
alldata =  pd.read_csv('./data/add_col_data/mergedata.csv', index_col=0) 
usedata =  alldata.drop(['the_total_time_need_to_fill_questionnaire_form',  
                         '0_if_features_5_and_8_are_equal', 'item_ID', 'how_frequently_the_user_eats_the_SUSHI'], axis=1)
# split 
train_df,test_df = train_test_split(usedata,test_size=0.2)
train_df,test_df = train_df.sort_values(by='user_ID') ,test_df.sort_values(by='user_ID') 


In [32]:
usedata.to_csv('./data/add_col_data/usedata.csv')

In [13]:
print('トレーニングデータ :',train_df.shape)
print('テストデータ : ' ,test_df.shape)

トレーニングデータ : (40000, 17)
テストデータ :  (10000, 17)


## catboost 

In [14]:
import catboost

In [15]:
train_data = train_df.drop(["user_ID", "score"], axis=1)
train_label = train_df["score"].values.astype(np.float32)

train_group_id = train_df["user_ID"].values.astype(np.float32) #.values#[ i for i in range(len(train_df["user_ID"].values))]

In [16]:
#catboost.Pool?

In [17]:
cat_features = [i for i in range(0,12)]
print(cat_features)
train_pool = catboost.Pool(data=train_data, label=train_label, group_id=train_group_id , cat_features=cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [18]:
test_data = test_df.drop(["user_ID", "score"], axis=1)
test_label = test_df["score"].values.astype(np.float32)
test_group_id = test_df["user_ID"].values.astype(np.float32)#[ i for i in range(len(test_df["user_ID"].values))]
test_pool = catboost.Pool(data=test_data, label=test_label, group_id=test_group_id, cat_features=cat_features)

In [19]:
param = {'loss_function':'YetiRank', 'learning_rate': .05, 'iterations': 200,
         'depth': 4, 'use_best_model':True}
model = catboost.CatBoost(param)
model.fit(train_pool, eval_set=test_pool)

0:	test: 0.9513635	best: 0.9513635 (0)	total: 481ms	remaining: 1m 35s
1:	test: 0.9500867	best: 0.9513635 (0)	total: 1.04s	remaining: 1m 43s
2:	test: 0.9518578	best: 0.9518578 (2)	total: 1.47s	remaining: 1m 36s
3:	test: 0.9524402	best: 0.9524402 (3)	total: 1.81s	remaining: 1m 28s
4:	test: 0.9515130	best: 0.9524402 (3)	total: 2.17s	remaining: 1m 24s
5:	test: 0.9551855	best: 0.9551855 (5)	total: 2.55s	remaining: 1m 22s
6:	test: 0.9527729	best: 0.9551855 (5)	total: 2.94s	remaining: 1m 21s
7:	test: 0.9531092	best: 0.9551855 (5)	total: 3.4s	remaining: 1m 21s
8:	test: 0.9531428	best: 0.9551855 (5)	total: 3.76s	remaining: 1m 19s
9:	test: 0.9535693	best: 0.9551855 (5)	total: 4.16s	remaining: 1m 18s
10:	test: 0.9502795	best: 0.9551855 (5)	total: 4.57s	remaining: 1m 18s
11:	test: 0.9502795	best: 0.9551855 (5)	total: 4.98s	remaining: 1m 18s
12:	test: 0.9502795	best: 0.9551855 (5)	total: 5.39s	remaining: 1m 17s
13:	test: 0.9533996	best: 0.9551855 (5)	total: 5.78s	remaining: 1m 16s
14:	test: 0.95314

117:	test: 0.9570912	best: 0.9575642 (92)	total: 51.6s	remaining: 35.8s
118:	test: 0.9570912	best: 0.9575642 (92)	total: 51.9s	remaining: 35.3s
119:	test: 0.9570912	best: 0.9575642 (92)	total: 52.3s	remaining: 34.9s
120:	test: 0.9569769	best: 0.9575642 (92)	total: 52.7s	remaining: 34.4s
121:	test: 0.9568896	best: 0.9575642 (92)	total: 53.1s	remaining: 33.9s
122:	test: 0.9568896	best: 0.9575642 (92)	total: 53.4s	remaining: 33.4s
123:	test: 0.9569232	best: 0.9575642 (92)	total: 53.8s	remaining: 33s
124:	test: 0.9579242	best: 0.9579242 (124)	total: 54.2s	remaining: 32.5s
125:	test: 0.9587268	best: 0.9587268 (125)	total: 54.5s	remaining: 32s
126:	test: 0.9590225	best: 0.9590225 (126)	total: 55s	remaining: 31.6s
127:	test: 0.9589939	best: 0.9590225 (126)	total: 55.3s	remaining: 31.1s
128:	test: 0.9589939	best: 0.9590225 (126)	total: 55.7s	remaining: 30.6s
129:	test: 0.9591883	best: 0.9591883 (129)	total: 56s	remaining: 30.2s
130:	test: 0.9589026	best: 0.9591883 (129)	total: 56.4s	remaining:

In [23]:
pred_score = model.predict(test_data)

array([ 0.31279626, -0.57645771,  0.34523635, ..., -0.42156022,
        0.06092889,  0.04343528])